In [ ]:
# importar paquetes necesarios
import os
import pandas as pd
import spacy
from spacy.kb import KnowledgeBase 
import numpy as np
import pandas as pd
import spacy
from csv import DictWriter
from timeit import default_timer as timer
import numpy as np

In [ ]:
# descargar modelo a utilizar para crear el knowledge Base
nlp = spacy.load("en_core_web_sm")
kb = KnowledgeBase(vocab=nlp.vocab, entity_vector_length=96)

In [ ]:
# obtener pablabras que se pueden eliminar
stopwords = nlp.Defaults.stop_words

In [ ]:

# obtener PT entities de dataframe
def getPTEntities(dataframe):
    
    PTList = dataframe[['Code_y','PT']]
    PTList = PTList.rename(columns = {'Code_y':'labels', 'PT':'entities'})
    
    return PTList.drop_duplicates()
    

# cargar conocimiento base
def loadKB(desc_dict):
    for qid, desc in desc_dict.items():
        desc_doc = nlp(desc)       
        desc_enc = desc_doc.vector
        kb.add_entity(entity=str(qid), entity_vector=desc_enc, freq=94)   # 342 is an arbitrary value here

# cargar alias para entidades
def loadAlias(name_dict):
    for qid, name in name_dict.items():
        kb.add_alias(alias=name, entities=[str(qid)], probabilities=[1])   # 100% prior probability P(entity|alias)


# obtener entitidades de fuzzy y LLT que van a ser los alias de los PT
def getEntitiesFuzzyAndLLT(entities):
    synDict = {}
    
    
    entities_group = entities.groupby("entities")

    for name, group in entities_group:
        synDict[name] = group['labels'].to_list()
        if len(group['labels'].to_list()) > 1:
            print(name)


    return synDict

# Crear diccionarios que seran utilizados para formar el conocimiento base
def getEntities(mainEntities):
    names = dict()
    descriptions = dict()
    
    for index, row in mainEntities.iterrows():
        qid, name, desc   = row['labels'], row['entities'], row['entities']
        names[qid]        = name
        descriptions[qid] = desc
    
    return names, descriptions

In [ ]:
# cargar tanto los LLT como las entidades encontradas con fuzzy
LLT_entidades = pd.read_csv("entidades.csv")

#obtener las entidades de MedDRA
df = pd.read_csv('MedDRACodes.csv')

#obtener PT principales
PTEntities = getPTEntities(df)

#almacenar entidades encontradas
PTEntities.to_csv("pt_entities_KB_v2.csv")

In [ ]:
# crear diccionarios para cargar conocimiento base
name_dict,desc_dict = getEntities(PTEntities)

In [ ]:
# cargar entidades principales y los alias iniciales
loadKB(desc_dict)
loadAlias(name_dict)

In [ ]:
# crear diccionario con los LLT y fuzzy matching
synDict = getEntitiesFuzzyAndLLT(LLT_entidades)

In [ ]:
# cargar alias con LLT y fuzzy, hay que realizar calculo de probabilidades
for k in synDict:
    probs = [1/len(synDict[k]) for qid in synDict[k]]
    arrays = np.apply_along_axis(lambda y: [str(i) for i in y], 0, synDict[k]).tolist()
    kb.add_alias(alias=k, entities=arrays, probabilities=probs)  # sum([probs]) should be <= 1 !

In [ ]:
# mostrar cantidatos para fever
print( [c.entity_ for c in kb.get_alias_candidates('fever')])

In [ ]:
#almacenar en disco
kb.to_disk("nel_kb_v3") 